- take input sequences and make blast database
- create pairwise combos by index

Alternatively:
- take pairwise sequences
- make blast database with unique seqs

Now have blast database and pairwise sequences (need to assign indices)

- Run Blast via iterator?
- Generate feature set for model
- Send to model

In [223]:
import os
import sys
import re
import shutil
import pandas as pd
import numpy as np
import tempfile

from typing import Collection

from itertools import combinations
import io

from Bio.Blast.Applications import NcbimakeblastdbCommandline, NcbiblastpCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

# https://biopython.org/docs/1.75/api/Bio.pairwise2.html
from Bio import pairwise2
from Bio.Align import substitution_matrices

In [2]:
s50k = pd.read_csv('../learn2therm_sample_50k_exploration.csv')

In [96]:
s50k['thermo_index'][0]

875

In [4]:
pd.set_option('display.max_columns', 40)
s50k.describe()

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,meso_index,prot_pair_index,meso_protein_int_index,thermo_protein_int_index,taxa_pair_index,local_gap_compressed_percent_id_16s,scaled_local_query_percent_id_16s,scaled_local_symmetric_percent_id_16s,query_align_cov_16s,subject_align_cov_16s,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_len,t_protein_len
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,5.000000e+04,5.000000e+04,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,24999.500000,0.349026,0.276478,0.270792,206.011660,0.791507,205.414920,0.770875,230.63206,8296.524160,8098.858120,9.105934e+07,3.213584e+07,2.600952e+07,362827.905140,0.906828,0.903266,0.904959,0.997153,0.997756,1061.404480,27.348365,51.505410,24.157045,269.985200,261.367700
std,14433.901067,0.096608,0.114404,0.116039,64.260901,0.144893,64.430548,0.179955,186.44859,5181.817228,4608.982941,5.233883e+07,1.868143e+07,1.662125e+07,229264.926253,0.015240,0.015626,0.015300,0.007108,0.003933,75.659661,3.737201,3.816972,3.586797,65.935753,69.675548
min,0.000000,0.164103,0.081301,0.045977,19.000000,0.284553,19.000000,0.082019,35.00000,37.000000,5.000000,1.219000e+03,8.554400e+04,3.063480e+05,4885.000000,0.876590,0.836055,0.880026,0.871051,0.866066,891.000000,7.000000,41.000000,20.000000,30.000000,27.000000
25%,12499.750000,0.293750,0.204545,0.202020,172.000000,0.673288,172.000000,0.656854,118.00000,3841.000000,4121.000000,4.645832e+07,1.619813e+07,1.628730e+07,173976.750000,0.895995,0.893020,0.893531,0.996728,0.996719,1009.000000,27.000000,50.000000,22.000000,233.000000,226.000000
50%,24999.500000,0.325000,0.250000,0.244648,210.000000,0.811321,209.000000,0.806584,186.00000,7134.000000,8046.000000,9.160212e+07,3.248324e+07,2.353333e+07,312362.000000,0.905983,0.902326,0.903543,0.997978,0.998009,1056.000000,28.000000,52.500000,24.500000,267.000000,260.000000
75%,37499.250000,0.367257,0.308300,0.301639,240.000000,0.923077,240.000000,0.923077,267.00000,14062.000000,12103.000000,1.360713e+08,4.819069e+07,2.868282e+07,617814.000000,0.916172,0.912576,0.914698,0.999350,0.999350,1108.000000,29.000000,52.500000,24.500000,317.000000,311.000000
max,49999.000000,0.985075,0.985075,0.985075,400.000000,1.000000,399.000000,1.000000,1788.00000,16507.000000,16564.000000,1.814866e+08,6.542306e+07,6.476579e+07,744518.000000,0.999351,0.993544,0.996439,1.000000,1.000000,1679.000000,40.000000,78.000000,56.000000,400.000000,400.000000


In [5]:
s50k['m_protein_seq'][:10].to_csv('single_list_test.csv', index = False)
s50k[['m_protein_seq', 't_protein_seq']][:10].to_csv('pair_list_test.csv', index = False)

In [6]:
slist = pd.read_csv('single_list_test.csv')

In [9]:
class FafsaPairs:
    
    def __init__(self, path):
        
        self.pairs = np.array(pd.read_csv(path))
        
    def align(self):
        
    
        

        
        
# Class for single sequence list input. Inherits from FAFSA_paired but is able to parse a single column input.
class FafsaSingles(FafsaPairs):
    
    def __init__(self, path):
        
        self.pairs = list(combinations(pd.read_csv(path).values, 2))


IndentationError: expected an indented block after function definition on line 7 (60237676.py, line 15)

In [10]:
test = FafsaPairs('pair_list_test.csv')
test.pairs

NameError: name 'FafsaPairs' is not defined

In [118]:
class BlastFiles:
    """Temporary files for use with BLAST CLI.
    
    Blast expects two input FASTA and produces an XML. The FASTA are redundant to CSV
    we already have. These files are created for the context and removed after completion.

    Parameters
    ----------
    query_iterator : iterator of (seq id, sequence)
        sequences to be used as query
    subject_iterator : iterator of (seq id, sequence)
        sequences to be used as the "database"

    Returns
    -------
    query_filename : str, name of fasta file with query sequences
    subject_filename : str, name of fasta file with subject sequences
    output_filename : str, name of output file for blast to save reuslts, will be deleted out of context
    """
    def __init__(self, query_iterator, subject_iterator, dbtype: str = 'nucl', dev_sample_num: int = None):
        # we have to create the temporary fasta files
        logger.debug("Creating temporary files to deposit blast inputs and outputs.")
        os.makedirs('./tmp/', exist_ok=True)
        query_temp = tempfile.NamedTemporaryFile('w', delete=False, dir='./tmp/')
        logger.debug(f"query file: {query_temp.name}")
        if dev_sample_num is not None:
            logger.debug(f"Using only max {dev_sample_num} sequences from query and subject")
        self.qt = query_temp.name
        n = 0
        for id_, seq in query_iterator:
            if seq == 'None' or seq is None:
                continue
            query_temp.write(f">{id_}\n{seq}\n")
            n +=1
            if dev_sample_num is not None:
                if n >= dev_sample_num:
                    break
        query_temp.close()
        logger.debug(f"added {n} sequences to query file")

        # folder for subject DB after we make a fasta
        subject_folder = tempfile.mkdtemp(dir='./tmp/')
        self.st = subject_folder
        subject_fasta_file = subject_folder+'/subs.fasta'
        self.subject_fasta_file = subject_fasta_file
        n = 0
        file = open(subject_fasta_file, 'w')
        for id_, seq in subject_iterator:
            if seq == 'None' or seq is None:
                continue
            file.write(f">{id_}\n{seq}\n")
            n +=1
            if dev_sample_num is not None:
                if n >= dev_sample_num:
                    break
        file.close()
        logger.debug(f"added {n} sequences to db file")

        # create db for it
        NcbimakeblastdbCommandline(dbtype=dbtype, input_file=subject_fasta_file, parse_seqids=True)()
        logger.debug(f"created database")
        # create the output xml file
        out_temp = tempfile.NamedTemporaryFile('w', delete=False, dir='./tmp')
        self.ot = out_temp.name

    def __enter__(self):
        return self.qt, self.subject_fasta_file, self.ot

    def __exit__(self, type, value, traceback):
        logger.debug("Removing temporary files used by blast")
        os.remove(self.qt)
        shutil.rmtree(self.st)
        os.remove(self.ot)

def csv_id_seq_iterator(csv_filepath: str, seq_col: str, index_col: str=None, id_filter: Collection = None, chunksize: int = 512, max_seq_length: int=None, **kwargs):
    """Returns a one by one iterator of seq ids and sequences to avoid OOM.
    
    Parameters
    ----------
    csv_filepath : str
        path to file containing data
    seq_col : str
        name of column containing sequences
    index_col: str, default None
        which column name is associated with the index, otherwise the 0th column will be used
    id_filter : Collection, Optional
        If given, only return sequences with the provided indexes
    chunksize : int, default 512
        Number of sequences that will be stored in memory at once.
    max_seq_length : int, default None
        Maximum length of sequence to return
    **kwargs passed to pandas read csv 
    """
    # first get the row numbers to consider
    if id_filter is not None:
        if index_col is not None:
            # get column positions to figure out which full col to load into memory
            columns = pd.read_csv(csv_filepath, nrows=1, **kwargs).columns
            index_col_position = np.argwhere(columns==index_col)[0][0]
            # load only that column
            row_indexes = pd.read_csv(csv_filepath, usecols=[index_col_position], **kwargs)
            row_indexes = pd.Series(row_indexes.set_index(index_col, drop=True).index)
        else:
            row_indexes = pd.read_csv(csv_filepath, usecols=[0]).index # just take the first column becuase we only need the indexes
            row_indexes = pd.Series(index=row_indexes, data=row_indexes)
        row_indexes_to_keep_mask = row_indexes.isin(id_filter)
        skiprows = lambda row_num: False if row_num==0 else not row_indexes_to_keep_mask.loc[row_num-1]
        logger.debug(f"{row_indexes_to_keep_mask.sum()} viable sequences in in file to iterate")
        seq_index_iterator = iter(list(row_indexes[row_indexes_to_keep_mask].values))
    else:
        skiprows=None

    for i, df_chunk in enumerate(pd.read_csv(csv_filepath, chunksize=chunksize, skiprows=skiprows, dtype=str, **kwargs)):
        if index_col is not None:
            df_chunk = df_chunk.set_index(index_col, drop=True)
        chunk = df_chunk[seq_col]
        logger.debug(f'Iterating chunk {i} seq in {csv_filepath}')
        for id_, seq in chunk.items():
            # skip long sequences
            if max_seq_length and len(seq) > max_seq_length:
                continue
            # in the case that there were no id filters, the id in the chunk corresponds to the correct sequence id
            # but if there was a filter, many rows were skipped and the indexes got jumbled, so we have to recapitulate
            # the correct seq index
            if id_filter is not None:
                yield next(seq_index_iterator), seq
            else:
                yield id_, seq


In [75]:
s50k.sample(2).to_csv('s5.csv')

In [38]:
metrics.compute_metric('bit_score')

,query_id,subject_id,bit_score


In [62]:
mesos = pd.read_csv('s5.csv')[['m_protein_seq', 'meso_index']]

In [ ]:
NcbimakeblastdbCommandline(dbtype='prot', input_file=subject_fasta_file, parse_seqids=True)()


In [82]:
test = open('test.fasta','w')

In [83]:
for row in mesos.iterrows():
    test.write(f">{row[1]['meso_index']}\n{row[1]['m_protein_seq']}\n")

In [92]:
cmd = NcbimakeblastdbCommandline(dbtype='prot', input_file='test.fasta', parse_seqids=True)()


In [98]:
NcbiblastpCommandline(query='test_q.fasta', db='test.fasta', outfmt=5, out='test_out', word_size=3, evalue=10000, max_target_seqs=100000)()


('', '')

In [238]:
class BlastMet:
    """Handles computation of metrics for each alignment in a blast record.

    The HSP with the largest average sequence coverage is used for local metrics.
    
    Parameters
    ----------
    blast_record : the record containing all hits for a query
    """
    def __init__(self, blast_record):
        recs = []
        for rec in blast_records:
            recs.append(rec)           # This is clunky
        self.record = recs[0]
        self.qid = self.record.query.split(' ')[0]

    def id_hsp_best_cov(self, alignment):
        """Determine HSP with the most average coverage of both sequences.
        
        Returns
        -------
        Index of HSP with max average seq coverage
        Max average coverage
        """
        scores = []
        #for alignment in self.record.alignments:
        for hsp in alignment.hsps:
            scores.append(
                ((hsp.query_end +1 - hsp.query_start)/self.record.query_length + (hsp.sbjct_end +1 - hsp.sbjct_start)/
                 alignment.length)/2)
        return np.argmax(scores), max(scores)
    
    @staticmethod
    def raw_gap_compressed_percent_id(n_matches, n_gaps, n_columns, n_compressed_gaps):
        """Percent matches in sequence, including but compressing gaps.
        
        Parameters
        ----------
        n_matches : int, number of matches in match columns
        n_gaps : number of gaps in match columns
        n_columns : total number of alignment match columns
        n_compressed_gaps : number of compressed gaps in match columns
        """
        return n_matches / (n_columns - n_gaps + n_compressed_gaps)
    
    def compute_metric(self,metric_name: str):
        """Compute the metric with specified name for each alignment"""
        if not hasattr(self, metric_name):
            raise ValueError(f"No metric found with name : {metric_name}")
        else:
            metric = getattr(self, metric_name)

        outputs = []
        for alignment in self.record.alignments:
            hsp_id, _ = self.id_hsp_best_cov(alignment)
            hsp = alignment.hsps[hsp_id]
            outputs.append((self.qid, alignment.hit_id.split('|')[-1], metric(alignment, hsp)))
        return pd.DataFrame(data=outputs, columns=['query_id', 'subject_id', metric_name])

    def subject_align_cov(self, alignment, hsp):
        """Fraction of AA on query string taken up by alignment"""
        return (hsp.sbjct_end +1 - hsp.sbjct_start)/alignment.length
    
    def subject_align_len(self, alignment, hsp):
        """Length of AA on query string taken up by alignment"""
        return int(hsp.sbjct_end +1 - hsp.sbjct_start)

    def query_align_cov(self, alignment, hsp):
        """Fraction of AA on query string taken up by alignment"""
        return (hsp.query_end +1 - hsp.query_start)/self.record.query_length

    def query_align_len(self, alignment, hsp):
        """Length of AA on query string taken up by alignment"""
        return int(hsp.query_end +1 - hsp.query_start)

    def bit_score(self, alignment, hsp):
        return hsp.score

    def local_gap_compressed_percent_id(self, alignment, hsp):
        """Percent matches in match sequence, including but compressing gaps.
        
        The largest local HSP score is used
        """
        n_matches = hsp.identities
        n_gaps = hsp.gaps
        n_columns = len(hsp.query)
        n_compressed_gaps = len(re.findall('-+', hsp.query))+len(re.findall('-+', hsp.sbjct))
        return self.raw_gap_compressed_percent_id(n_matches, n_gaps, n_columns, n_compressed_gaps)

    def scaled_local_query_percent_id(self, alignment, hsp):
        """Percent matches in query sequence based on best HSP."""
        return hsp.identities/self.record.query_length

    def scaled_local_symmetric_percent_id(self, alignment, hsp):
        """Percent matches compared to average seq length of query and subject based on best HSP"""
        return 2*hsp.identities/(self.record.query_length + alignment.length)

In [241]:
result_handle = open('test_out')
blast_records = NCBIXML.parse(result_handle)
test = BlastMet(blast_records)

for m in [
    'local_gap_compressed_percent_id',
    'scaled_local_query_percent_id',
    'scaled_local_symmetric_percent_id',
    'query_align_len',
    'query_align_cov',
    'subject_align_len',
    'subject_align_cov',
    'bit_score'
]: 
    print(test.compute_metric(m))

  query_id subject_id  local_gap_compressed_percent_id
0      875       6565                         0.296296
1      875       5029                         0.367347
2      875       8505                         0.384615
3      875       2327                         0.214286
4      875      15825                         0.259259
5      875       6801                         0.197368
6      875       7923                         0.444444
7      875       2373                         0.250000
8      875      11651                         0.307692
9      875       6824                         0.269231
  query_id subject_id  scaled_local_query_percent_id
0      875       6565                       0.079208
1      875       5029                       0.089109
2      875       8505                       0.049505
3      875       2327                       0.029703
4      875      15825                       0.034653
5      875       6801                       0.074257
6      875       7923   